In [5]:
# -*- coding: utf-8 -*-
"""
Created on Sat Dec 21 17:51:31 2019

@author: Abdulrahman Alothman
"""

import numpy as np # linear algebra
import pandas as pd # data processing

rating = pd.read_csv('rating.csv',encoding = "latin-1")
anime = pd.read_csv('anime.csv')

In [6]:
anime = anime.astype({"name":'category',"genre":'category',"episodes":'category','type':'category' })
print(anime.isnull().sum().to_frame('nulls'))

#eksik veriler doldurulur
anime.rating=anime.rating.fillna(anime.rating.mean())
anime["genre"]=(anime["genre"].cat.codes.replace(-1, np.nan).interpolate().astype(int).astype('category').cat.rename_categories(anime["genre"].cat.categories))
anime["type"]=(anime["type"].cat.codes.replace(-1, np.nan).interpolate().astype(int).astype('category').cat.rename_categories(anime["type"].cat.categories))
print(anime.isnull().sum().to_frame('nulls'))


#iki dosyayı birleştir ve -1 leri sil 
df_rating1 = pd.merge(rating,anime.drop('rating',axis=1),on='anime_id')
df_rating=df_rating1[df_rating1.rating!=-1]
df_rating.head(7)

          nulls
anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
          nulls
anime_id      0
name          0
genre         0
type          0
episodes      0
rating        0
members       0


,user_id,anime_id,rating,name,genre,type,episodes,members
1,3,20,8,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
2,5,20,6,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
5,21,20,8,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
6,28,20,9,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
7,34,20,9,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
8,38,20,6,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
9,39,20,10,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297


In [7]:
#her filmin aldığı rating değerinin otralaması ,değerlendiren kişi sayısı ,anime_id
ratings = pd.DataFrame(df_rating.groupby('name')['rating'].mean())
ratings['vote_count'] = pd.DataFrame(df_rating.groupby('name')['rating'].count()) #dataFrame vote_count sütünü eklemek
ratings['anime_id']=pd.DataFrame(df_rating.groupby('name')['anime_id'].unique())

ort=ratings['rating'].mean() #tüm filmlerin rating ortalamaların ortalaması 
m= ratings['vote_count'].quantile(0.9) #bir filmin listede yer alması için listedeki filmlerin en az% 90'ından daha
                                       # fazla oy alması gerekir.
q_filmler=ratings.copy().loc[ratings['vote_count']>=m] #listede yer alacak filmler 1302.8 den fazla oy alması gerkir
q_filmler.shape

(1230, 3)

In [9]:
#Bu listede yer almaya hak kazanan 1230 film olduğunu görüyoruz.

def weighted_rating(x, m=m, ort=ort):
    v = x['vote_count']
    R = x['rating']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * ort)

#fon. listeye uygulaması
q_filmler['score'] = q_filmler.apply(weighted_rating, axis=1)
#Sort movies based on score calculated above

q_filmler = q_filmler.sort_values('score', ascending=False)
#Print the top 15 movies
q_filmler[['anime_id', 'vote_count', 'rating', 'score']].head(10)

,anime_id,vote_count,rating,score
name,,,,
Fullmetal Alchemist: Brotherhood,[5114],21494,9.322741,9.169285
Steins;Gate,[9253],17151,9.261326,9.076090
Clannad: After Story,[4181],15518,9.165292,8.969512
Code Geass: Hangyaku no Lelouch R2,[2904],21124,9.059837,8.919122
Hunter x Hunter (2011),[11061],7477,9.234586,8.849219
Code Geass: Hangyaku no Lelouch,[1575],24126,8.927796,8.810458
Sen to Chihiro no Kamikakushi,[199],19481,8.923259,8.779982
Tengen Toppa Gurren Lagann,[2001],16955,8.933176,8.769369
Death Note,[1535],34226,8.834629,8.754064
